# Data Science Workshop: Web Scraping & MySQL Integration

Welcome, Data Science students! In this workshop, we'll dive into practical data acquisition techniques: web scraping and interacting with databases. Our goal is to gather information about NFL teams, their conferences, divisions, personnel, and championships, store it in a structured way, and then persist it in a MySQL database.

## Learning Objectives:
1.  **Web Scraping:** Extract structured data from websites using `requests` and `BeautifulSoup`.
2.  **Data Manipulation:** Organize scraped data into Pandas DataFrames.
3.  **Function Design:** Create reusable functions for data extraction and updates.
4.  **Database Interaction:** Understand SQL schema, create tables, and push/pull data using Python's `sqlalchemy` and Pandas `to_sql`/`read_sql`.
5.  **Data Integration:** Link related data across tables using primary and foreign keys.

**Important Note:** If you encounter issues, please ensure your `keys.py` file is correctly configured with your MySQL password and that your MySQL server is running before executing the database connection and table creation steps.

## Part 1: Setting Up Environment

First, we'll import the necessary libraries required for web scraping, data manipulation, and database interaction.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re

from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError, ProgrammingError

import keys

## Part 2: Implementing Helper Functions

In this section, you will implement several helper functions. These functions will be crucial for abstracting the logic for web scraping specific types of data and for pushing DataFrames to our MySQL database. Building these functions step-by-step will prepare you for the main data pipeline.

### Task 2.1: Implement `get_nfl_teams_from_wiki`

Create a Python function named `get_nfl_teams_from_wiki`. This function should:
1.  Define the URL for the National Football League Wikipedia page.
2.  Fetch the HTML content using `requests` and parse it with `BeautifulSoup`.
3.  Locate the table containing NFL team information. This table can often be identified by its caption (e.g., "National Football League teams") or by checking for specific classes and headers within candidate tables.
4.  Parse the table, handling `rowspan` and `colspan` attributes to correctly reconstruct the grid of data.
5.  Extract the 'conference', 'division', and 'team_name' names from the parsed table.
6.  Clean the extracted team names (e.g., remove footnote symbols like `[*]`, `[†]`, and text within square brackets `[1]`).
7.  Return a Pandas DataFrame with columns 'conference', 'division', and 'team_name'. This DataFrame will serve as the source for both the `teams` and `conferences` tables in our database.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `get_nfl_teams_from_wiki`:

**Step 1: Define URL and Fetch Content**
* Set `url` to `"https://en.wikipedia.org/wiki/National_Football_League"`.
* Use `requests.get(url)` and `response.raise_for_status()` for error handling.
* Parse the content with `BeautifulSoup(response.text, 'html.parser')`.

**Step 2: Locate the NFL Teams Table**
* Try to find the table by its `<caption>` tag, looking for text like "National Football League teams" (use `re.compile` with `re.IGNORECASE` for flexibility).

**Step 3: Handle `rowspan` and `colspan` to Build a Grid**
* Initialize a 2D list (grid) to represent the table, filled with `None`. The number of columns can be determined by summing `colspan` values from the first header row.
* Iterate through each `<tr>` (HTML row) in the `nfl_teams_table`.
* For each `<th>` or `<td>` (HTML cell) in the current `<tr>`:
    * Determine its `rowspan` and `colspan` attributes (default to 1 if not present).
    * Find the next available `c_idx` (column index) in your `grid` for the current `r_idx` (row index), skipping cells already filled by a previous span.
    * Fill the corresponding cells in your `grid` with the `cell_value` for the extent of its `rowspan` and `colspan`.
    * Advance `c_idx` by the `colspan` value.

**Step 4: Extract Data from the Grid**
* The actual data rows usually start after the header rows (e.g., from `grid[1]` onwards).
* For each data row in your `grid`:
    * 'Conference' is at `row_values[0]`.
    * 'Division' is at `row_values[1]`.
    * 'Team' name position is at `row_values[2]`
* **Clean Team Names:** Use `re.sub(r'\[.*?\]|\*|\†', '', team_name).strip()` to remove footnotes and other unwanted characters.

**Step 5: Return DataFrame**
* Append dictionaries containing 'Conference', 'Division', and 'Team' to a list.
* Convert this list of dictionaries into a Pandas DataFrame.
</details>

##### Solution

In [ ]:
def get_nfl_teams_from_wiki():
    """
    Scrapes the NFL teams table from Wikipedia to extract Conference, Division, and Team names.
    Returns a Pandas DataFrame with 'conference', 'division', and 'team_name' columns.
    """
    url = "https://en.wikipedia.org/wiki/National_Football_League"
    teams_final_data = []

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        caption = soup.find('caption', string=re.compile(r'National Football League teams', re.IGNORECASE))
        nfl_teams_table = caption.find_parent('table')

        if nfl_teams_table is None:
            print("Error: Could not find the NFL teams table on the page.")
            return pd.DataFrame()
        else:
            table_rows = nfl_teams_table.find_all('tr')

            # Determine the number of columns by summing colspan values from the first header row
            num_cols = sum(int(th.get('colspan', 1)) for th in table_rows[0].find_all('th'))

            # Initialize a grid to handle rowspan and colspan
            grid = [[None for _ in range(num_cols)] for _ in range(len(table_rows))]

            for r_idx, row_html in enumerate(table_rows):
                c_idx = 0

                for cell_html in row_html.find_all(['th', 'td']):
                    # Skip cells already filled by a previous span
                    while c_idx < num_cols and grid[r_idx][c_idx] is not None:
                        c_idx += 1

                    if c_idx >= num_cols:
                        break

                    rowspan = int(cell_html.get('rowspan', 1))
                    colspan = int(cell_html.get('colspan', 1))
                    cell_value = re.sub(r'\s+', ' ', cell_html.get_text(strip=True)).strip()

                    # Fill the grid with the cell value for the extent of its rowspan and colspan
                    for i in range(r_idx, r_idx + rowspan):
                        for j in range(c_idx, c_idx + colspan):
                            if i < len(grid) and j < num_cols:
                                grid[i][j] = cell_value

                    c_idx += colspan

            # Extract data from the grid, skipping header rows (assuming data starts from row 1)
            for r_idx in range(1, len(grid)):
                row_values = grid[r_idx]

                # Assign values based on expected column positions
                conference = row_values[0]
                division = row_values[1]
                team = row_values[2]

                # Clean team name
                if team:
                    team = re.sub(r'\[.*?\]|\*|\†', '', team).strip()

                teams_final_data.append({
                    'conference': conference,
                    'division': division,
                    'team_name': team
                })

            df = pd.DataFrame(teams_final_data)
            return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return pd.DataFrame()

### Task 2.2: Implement `get_static_team_data`

Create a Python function named `get_static_team_data` that takes a `team_name` as input. This function should:
1.  Construct the Wikipedia URL for the given team.
2.  Use `requests` to fetch the page content and `BeautifulSoup` to parse it.
3.  Extract the 'Location', 'Stadium', 'Established', 'Colors', 'Mascot', and 'Website' from the team's infobox.
4.  Clean the extracted text (e.g., remove `[\d+]` references).
5.  Return a dictionary containing the scraped information, with keys directly matching the column names in the `teams` SQL table.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `get_static_team_data`:

**Step 1: Construct the URL and Fetch Content**
* Start by replacing spaces in `team_name` with underscores to form the Wikipedia URL (e.g., "New England Patriots" becomes "New_England_Patriots").
* Use `requests.get()` to fetch the HTML content of the Wikipedia page.
* Parse the HTML content using `BeautifulSoup(response.text, 'html.parser')`.

**Step 2: Locate the Infobox**
* Most Wikipedia pages for entities like sports teams have a main information box on the right side. This is typically a `<table>` element with the class `infobox`.
* Use `soup.find_all('table', class_='infobox')` to find all infobox tables. You'll need to iterate through these to find the *main* one, which often contains a `<th>` (table header) with text like 'Basic info' or 'Personnel'.

**Step 3: Extract Data from Infobox Rows**
* Once you've identified the correct `infobox` table, iterate through its `<tr>` (table row) elements.
* Within each `<tr>`, look for `<th>` (header) and `<td>` (data) tags. The `<th>` usually contains the label (e.g., 'Location', 'Stadium'), and the `<td>` contains the corresponding value.
* Use `.get_text(strip=True)` to extract clean text from these tags.

**Step 4: Clean and Store Specific Data Points with appropriate Keys**
* **General Cleaning:** For all extracted `data_text`, use `re.sub(r'\[\d+\]', '', data_text)` to remove citation numbers like `[1]`, `[2]`, etc.
* **'Established':** This field often contains a full date and age (e.g., "August 14, 1959; 65 years ago"). Use `re.search(r'\b\d{4}\b', data_text)` to extract just the four-digit year and convert it to an integer. Store this in the dictionary with the key `'established_year'`.
* **'Stadium' and 'Location':** These can be tricky. The `<td>` might contain both the stadium name and location, sometimes linked. Try to get the text of the first `<a>` tag within the `<td>` for the stadium. For location, you might need to extract text that follows the stadium name, or look for other `<a>` tags that represent cities/states. Clean up any extra parentheses or newline characters. Store these with keys `'stadium'` and `'city'`/`'state'` respectively.
* **'Colors':** This might be a simple text string or contain color swatches. Try to get the direct text content using `data.find(text=True, recursive=False).strip()`. If that's empty, you might need to combine text from child `<a>` tags, stopping if you encounter a `<br>` tag. Store this with the key `'colors'`.
* **'Mascot' and 'Website':** These are usually straightforward text extractions. For 'Website', look for an `<a>` tag with `class_='external text'` and extract its `href` attribute. Store these with keys `'mascot'` and `'website'`.
* **`team_name`:** Ensure you include the original `team_name` input as `'team_name'` in the dictionary.

**Step 5: Return the Dictionary**
* Populate your `static_info` dictionary with the extracted and cleaned values, ensuring all keys match your SQL schema.
</details>

##### Solution

In [ ]:
def get_static_team_data(team_name):
    """
    Scrapes static information (City, State, Stadium, Established, Colors, Mascot, Website)
    for a given NFL team from its Wikipedia page.
    """
    url_team_name = team_name.replace(' ', '_')
    url = f"https://en.wikipedia.org/wiki/{url_team_name}"

    static_info = {
        'team_name': team_name,
        'city': None,
        'state': None,
        'stadium': None,
        'established_year': None,
        'colors': None,
        'mascot': None,
        'website': None
    }

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return static_info

    infobox = None
    infoboxes = soup.find_all('table', class_='infobox')
    for ibox in infoboxes:
        # Look for infoboxes that contain 'Basic info' or 'Personnel' headers
        if ibox.find('th', class_='infobox-header', string='Basic info') or \
           ibox.find('th', class_='infobox-header', string='Personnel'):
            infobox = ibox
            break

    if not infobox:
        print(f"Could not find infobox for {team_name}")
        return static_info

    for row in infobox.find_all('tr'):
        header = row.find('th')
        data = row.find('td')
        if header and data:
            header_text = header.get_text(strip=True)
            data_text_cleaned = re.sub(r'\[\d+\]', '', data.get_text(strip=True))

            if 'Established' in header_text:
                match = re.search(r'\b\d{4}\b', data_text_cleaned)
                if match:
                    static_info['established_year'] = int(match.group(0))
            elif 'Stadium' in header_text:
                stadium_link = data.find('a')
                if stadium_link:
                    static_info['stadium'] = stadium_link.get_text(strip=True)

                full_td_text = data.get_text(strip=True)
                cleaned_td_text = re.sub(r'\[\d+\]', '', full_td_text).strip()

                if static_info['stadium'] and static_info['stadium'] in cleaned_td_text:
                    cleaned_td_text = cleaned_td_text.replace(static_info['stadium'], '', 1).strip()

                parentheses_match = re.search(r'\(([^)]*)\)', cleaned_td_text)
                if parentheses_match:
                    location_in_paren = parentheses_match.group(1).strip()
                    parts = [p.strip() for p in location_in_paren.split(',') if p.strip()]
                    if len(parts) >= 1:
                        static_info['city'] = parts[0]
                    if len(parts) >= 2:
                        static_info['state'] = parts[1]
                else:
                    parts = [p.strip() for p in cleaned_td_text.split(',') if p.strip()]
                    if len(parts) >= 1:
                        static_info['city'] = parts[0]
                    if len(parts) >= 2:
                        static_info['state'] = parts[1]

            elif 'Colors' in header_text:
                colors_text = None
                plainlist_div = data.find('div', class_='plainlist')
                if plainlist_div:
                    first_li = plainlist_div.find('li')
                    if first_li:
                        colors_text = re.sub(r'\[\d+\]', '', first_li.get_text(strip=True)).strip()

                colors_parts = []
                for content in data.contents:
                    if isinstance(content, str) and content.strip():
                        cleaned_part = re.sub(r'\[\d+\]', '', content.strip())
                        colors_parts.append(cleaned_part)
                    elif content.name == 'br': # Stop at line breaks
                        break
                    elif content.name == 'style' or (content.name == 'span' and 'legend-color' in content.get('class', [])):
                        # Skip style tags or color legend spans
                        break
                    elif content.name == 'sup': # Skip superscript (citations)
                        continue
                    elif content.name == 'a' and content.get_text(strip=True) and content.get_text(strip=True).lower() not in ['colors', 'color']:
                        colors_parts.append(content.get_text(strip=True))

                colors_text = ', '.join(filter(None, colors_parts)).strip()
                colors_text = colors_text.strip(', ') # Remove trailing comma if any

                static_info['colors'] = colors_text if colors_text else None

            elif 'Mascot' in header_text:
                static_info['mascot'] = data_text_cleaned
            elif 'Website' in header_text:
                website_link = data.find('a', class_='external text')
                if website_link:
                    static_info['website'] = website_link.get('href')

    return static_info


### Task 2.3: Implement `get_personnel_data`

Create a Python function named `get_personnel_data` that takes a `team_name` as input. This function should:
1.  Construct the Wikipedia URL for the given team.
2.  Scrape the 'Owner(s)', 'President', 'General manager', and 'Head coach' from the infobox.
3.  Add an `extraction_date` column with the current date.
4.  Return a dictionary with the extracted personnel data, with keys directly matching the column names in the `personnel` SQL table.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `get_personnel_data`:

**Step 1: Construct URL and Fetch Content**
* Similar to Task 2.1, form the Wikipedia URL and use `requests` and `BeautifulSoup` to get the parsed HTML.

**Step 2: Locate the 'Personnel' Infobox**
* Again, search for `<table>` elements with `class='infobox'`.
* This time, specifically look for the infobox that contains a `<th>` with the text 'Personnel'. This indicates you've found the correct section for personnel information.

**Step 3: Extract Personnel Details with appropriate Keys**
* Iterate through the `<tr>` elements within the identified 'Personnel' infobox.
* For each row, extract the text from the `<th>` (header) and `<td>` (data) tags.
* **Headers to look for:** 'Owner(s)', 'President', 'General manager', 'Head coach'. Be aware that 'General manager' might sometimes appear as 'GM'.
* **Clean Text:** Remember to use `re.sub(r'\[\d+\]', '', data_text)` to remove any citation numbers from the extracted data.
* Store the extracted values in your `personnel_info` dictionary with keys matching your SQL schema: `'owner'`, `'president'`, `'general_manager'`, `'head_coach'`.

**Step 4: Add `extraction_date`**
* Include a key-value pair in your `personnel_info` dictionary for `'extraction_date'`, setting its value to `datetime.now().strftime('%Y-%m-%d')`.

**Step 5: Return the Dictionary**
* Return the `personnel_info` dictionary with all the extracted details.
</details>

##### Solution

In [ ]:
def get_personnel_data(team_name):
    """
    Scrapes the Wikipedia page for a given NFL team to extract personnel information
    (Owner, President, Head Coach, General Manager) and adds the current extraction date.
    """
    url_team_name = team_name.replace(' ', '_')
    url = f"https://en.wikipedia.org/wiki/{url_team_name}"

    personnel_info = {
        'team_name': team_name,
        'owner': None,
        'president': None,
        'head_coach': None,
        'general_manager': None,
        'extraction_date': datetime.now().strftime('%Y-%m-%d')
    }

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return personnel_info

    infobox = None
    infoboxes = soup.find_all('table', class_='infobox')
    for ibox in infoboxes:
        if ibox.find('th', class_='infobox-header', string='Personnel'):
            infobox = ibox
            break

    if not infobox:
        print(f"Could not find personnel infobox for {team_name}")
        return personnel_info

    for row in infobox.find_all('tr'):
        header = row.find('th')
        data = row.find('td')
        if header and data:
            header_text = header.get_text(strip=True)
            data_text = re.sub(r'\[\d+\]', '', data.get_text(strip=True))

            if 'Owner(s)' in header_text:
                personnel_info['owner'] = data_text
            elif 'President' in header_text:
                personnel_info['president'] = data_text
            elif 'Head coach' in header_text:
                personnel_info['head_coach'] = data_text
            elif 'General manager' in header_text or 'GM' in header_text:
                personnel_info['general_manager'] = data_text

    return personnel_info


### Task 2.4: Implement `get_championship_data`

Create a Python function named `get_championship_data` that takes a Pandas DataFrame (`championships_df`) and a SQLAlchemy engine (`sql_engine`) as input. This function should:
1.  Construct the Wikipedia URL for the given team.
2.  Scrape the counts for 'League championships', 'Conference championships', and 'Division championships' from the infobox.
3.  Return a dictionary with the extracted championship data, with keys directly matching the column names in the `championships` SQL table. **Note: This function will NOT include `extraction_year` as the database will only store the latest championship data per team.**

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `get_championship_data`:

**Step 1: Construct URL and Fetch Content**
* As before, create the Wikipedia URL and fetch/parse the HTML.

**Step 2: Locate the 'Championships' Infobox Section**
* Find the `infobox` table that contains a `<th>` with the text 'Championships'. This section typically lists the counts for various championship types.

**Step 3: Extract Championship Counts with appropriate Keys**
* The championship details might not be in direct `<td>` siblings of the 'Championships' header. You might need to find the parent `<tr>` of the 'Championships' header and then iterate through subsequent sibling `<tr>` elements.
* Within each relevant row, use `re.search()` with patterns like `r'League championships:\s*(\d+)'`, `r'Conference championships:\s*(\d+)'`, and `r'Division championships:\s*(\d+)'` to find the numerical counts.
* Remember to convert the extracted digits (which will be strings) to integers using `int()`.
* Note that 'League championships' might sometimes be represented by 'Super Bowl championships' on some pages. You can use the Super Bowl count as a fallback if the general 'League championships' isn't found.
* Store the extracted counts in your `championship_info` dictionary with keys matching your SQL schema: `'league_championships'`, `'conference_championships'`, `'division_championships'`.

**Step 4: Return the Dictionary**
* Return the `championship_info` dictionary. **Crucially, do NOT include `extraction_year` in this dictionary, as the database will only store the latest championship data per team.**
</details>

##### Solution

In [ ]:
def get_championship_data(team_name):
    """
    Scrapes championship information (League, Conference, Division championships)
    for a given NFL team from its Wikipedia page.
    """
    url_team_name = team_name.replace(' ', '_')
    url = f"https://en.wikipedia.org/wiki/{url_team_name}"

    championship_info = {
        'team_name': team_name,
        'league_championships': None,
        'conference_championships': None,
        'division_championships': None,
    }

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return championship_info

    infobox = None
    infoboxes = soup.find_all('table', class_='infobox')
    for ibox in infoboxes:
        if ibox.find('th', class_='infobox-header', string='Championships'):
            infobox = ibox
            break

    if not infobox:
        return championship_info

    championships_header = infobox.find('th', class_='infobox-header', string='Championships')
    if championships_header:
        current_row = championships_header.find_parent('tr').find_next_sibling('tr')
        while current_row:
            row_text = current_row.get_text(strip=True)

            league_match = re.search(r'League championships:\s*(\d+)', row_text)
            if league_match:
                championship_info['league_championships'] = int(league_match.group(1))
            elif 'Super Bowl championships' in row_text:
                super_bowl_match = re.search(r'Super Bowl championships:\s*(\d+)', row_text)
                if super_bowl_match and championship_info['league_championships'] is None:
                    championship_info['league_championships'] = int(super_bowl_match.group(1))

            conference_match = re.search(r'Conference championships:\s*(\d+)', row_text)
            if conference_match:
                championship_info['conference_championships'] = int(conference_match.group(1))

            division_match = re.search(r'Division championships:\s*(\d+)', row_text)
            if division_match:
                championship_info['division_championships'] = int(division_match.group(1))

            next_row_candidate = current_row.find_next_sibling('tr')
            if next_row_candidate and next_row_candidate.find('th', class_='infobox-header'):
                break
            current_row = next_row_candidate

    return championship_info


### Task 2.5: Implement `push_teams_to_sql`

Create a Python function named `push_teams_to_sql` that takes a Pandas DataFrame (`df`) and a SQLAlchemy engine (`sql_engine`) as input. This function should:
1.  Push the DataFrame to the `teams` table in the MySQL database.
2.  Handle potential duplicate team names by updating existing records or inserting new ones using `INSERT ... ON DUPLICATE KEY UPDATE`.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `push_teams_to_sql` using `INSERT ... ON DUPLICATE KEY UPDATE`:

**Step 1: Iterate and Upsert**
* Iterate through each row of the input DataFrame `df`.
* For each row, construct an `INSERT` statement that includes an `ON DUPLICATE KEY UPDATE` clause.
* The `ON DUPLICATE KEY UPDATE` clause should specify which columns to update if a record with the same `team_name` (which is a `UNIQUE` key) already exists. Use `VALUES(column_name)` to refer to the new values being inserted.
* Execute this SQL statement using `connection.execute(text(...))` within a database connection.

**Step 2: Commit Changes**
* After iterating through all rows, commit the changes to the database using `connection.commit()`.
</details>

##### Solution

In [ ]:
def push_teams_to_sql(df, sql_engine):
    """
    Pushes the NFL teams DataFrame to the 'teams' table.
    Handles potential duplicates by updating existing team records or inserting new ones.
    """
    if df.empty:
        print("No team data to push to 'teams' table.")
        return

    try:
        with sql_engine.connect() as connection:
            for index, row in df.iterrows():
                team_name = row['team_name']
                city = row.get('city')
                state = row.get('state')
                stadium = row.get('stadium')
                established_year = row.get('established_year')
                colors = row.get('colors')
                mascot = row.get('mascot')
                website = row.get('website')

                sql_statement = text("""
                    INSERT INTO teams (team_name, city, state, stadium, established_year, colors, mascot, website)
                    VALUES (:team_name, :city, :state, :stadium, :established_year, :colors, :mascot, :website)
                    ON DUPLICATE KEY UPDATE
                    city = VALUES(city),
                    state = VALUES(state),
                    stadium = VALUES(stadium),
                    established_year = VALUES(established_year),
                    colors = VALUES(colors),
                    mascot = VALUES(mascot),
                    website = VALUES(website)
                """)
                connection.execute(sql_statement, {
                    'team_name': team_name,
                    'city': city,
                    'state': state,
                    'stadium': stadium,
                    'established_year': established_year,
                    'colors': colors,
                    'mascot': mascot,
                    'website': website
                })
            connection.commit()
            print(f"Successfully pushed/updated {len(df)} team records in 'teams' table.")
    except Exception as e:
        print(f"Error pushing teams to SQL: {e}")


### Task 2.6: Implement `push_conferences_to_sql`

Create a Python function named `push_conferences_to_sql` that takes a Pandas DataFrame (`conferences_df`) and a SQLAlchemy engine (`sql_engine`) as input. This function should:
1.  **Expect `team_id` to already be present in `conferences_df`.**
2.  Push the DataFrame to the `conferences` table in the MySQL database.
3.  Handle potential duplicates by only inserting new conference records or updating existing ones if the conference/division changed for a team.

##### Hint
<details>
<summary>Click to reveal hint</summary>

**Step 1: Prepare DataFrame**
* The input `conferences_df` should already have `team_id`, `conference`, and `division` columns. Ensure `team_id` is of integer type and handle any `NaN` values if necessary.

**Step 2: Implement Upsert Logic (Update or Insert)**
* Iterate through each row of your `conferences_df`.
* For each row:
    * Construct an `INSERT` statement with `ON DUPLICATE KEY UPDATE` clauses for `conference` and `division`. This will insert a new record if `team_id` doesn't exist, or update the `conference` and `division` if `team_id` already exists.
    * Execute the SQL statement using `connection.execute(text(...))`.
* Remember to `connection.commit()` after the loop to save all changes.
</details>

##### Solution

In [ ]:
def push_conferences_to_sql(conferences_df, sql_engine):
    """
    Pushes the NFL conferences DataFrame to the 'conferences' table.
    Updates existing records if conference/division data is unchanged, otherwise inserts new records.
    """
    try:
        final_conferences_df = conferences_df.copy()
        final_conferences_df.dropna(subset=['team_id'], inplace=True)
        final_conferences_df['team_id'] = final_conferences_df['team_id'].astype(int)

        if not final_conferences_df.empty:
            with sql_engine.connect() as connection:
                # Implement Upsert Logic (Update or Insert)
                for index, row in final_conferences_df.iterrows():
                    team_id = row['team_id']
                    conference = row['conference']
                    division = row['division']

                    sql_statement = text("""
                        INSERT INTO conferences (team_id, conference, division)
                        VALUES (:team_id, :conference, :division)
                        ON DUPLICATE KEY UPDATE
                        conference = VALUES(conference),
                        division = VALUES(division)
                    """)
                    connection.execute(sql_statement, {
                        'team_id': team_id,
                        'conference': conference,
                        'division': division
                    })
                connection.commit()
                print(f"Successfully pushed/updated {len(final_conferences_df)} conference records to 'conferences' table.")
        else:
            print("No conference data to push (or no matching teams found).")
    except Exception as e:
        print(f"Error pushing conferences to SQL: {e}")


### Task 2.7: Implement `push_personnel_to_sql`

Create a Python function named `push_personnel_to_sql` that takes a Pandas DataFrame (`personnel_df`) and a SQLAlchemy engine (`sql_engine`) as input. This function should:
1.  Retrieve `team_id` and `team_name` from the `teams` table in the database.
2.  Merge the `personnel_df` with the retrieved team information to add the `team_id`.
3.  Push the DataFrame to the `personnel` table in the MySQL database.
4.  Handle potential duplicates by inserting new records only if the personnel data for a team has changed for a given `extraction_date`.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `push_personnel_to_sql`:

**Step 1: Retrieve `team_id`s from the Database**
* Use `pd.read_sql('SELECT team_id, team_name FROM teams', sql_engine)` to get the necessary team IDs.
* Merge this DataFrame with your input `personnel_df` on `team_name` to associate each personnel record with its corresponding `team_id`.

**Step 2: Implement Intelligent Upsert Logic**
* Iterate through each row of the merged `personnel_df`.
* For each team, first check if a personnel record for that `team_id` already exists in the `personnel` table.
    * If a record exists, compare the current personnel data (owner, president, head coach, general manager) with the latest existing record for that team.
    * If *any* of the personnel fields have changed then insert a *new* record into the `personnel` table. This preserves historical changes.
    * If no record exists, simply insert the new record.
* Use `connection.execute(text(...))` for both the `SELECT` (to check for existing data) and `INSERT` statements.

**Step 3: Commit Changes**
* After the loop, `connection.commit()` to save all changes.
</details>

##### Solution

In [ ]:
def push_personnel_to_sql(personnel_df, sql_engine):
    """
    Pushes the personnel DataFrame to the 'personnel' table.
    Retrieves team_ids from the 'teams' table to establish foreign key links.
    Inserts new records only if personnel data for a team has changed for a given extraction_date.
    """
    if personnel_df.empty:
        print("No personnel data to push to 'personnel' table.")
        return

    try:
        teams_from_db = pd.read_sql('SELECT team_id, team_name FROM teams', sql_engine)
        personnel_df_with_ids = pd.merge(personnel_df,
                                         teams_from_db,
                                         left_on='team_name',
                                         right_on='team_name',
                                         how='inner')

        personnel_df_with_ids.dropna(subset=['team_id'], inplace=True)
        personnel_df_with_ids['team_id'] = personnel_df_with_ids['team_id'].astype(int)

        if not personnel_df_with_ids.empty:
            with sql_engine.connect() as connection:
                for index, row in personnel_df_with_ids.iterrows():
                    team_id = row['team_id']
                    owner = row.get('owner')
                    president = row.get('president')
                    head_coach = row.get('head_coach')
                    general_manager = row.get('general_manager')
                    extraction_date = row['extraction_date']

                    existing_personnel_query = text("""
                        SELECT owner, president, head_coach, general_manager, extraction_date
                        FROM personnel
                        WHERE team_id = :team_id
                        ORDER BY extraction_date DESC LIMIT 1
                    """)
                    result = connection.execute(existing_personnel_query, {'team_id': team_id}).fetchone()

                    should_insert = False
                    if result:
                        existing_owner, existing_president, existing_head_coach, existing_general_manager, existing_date = result

                        existing_date_str = existing_date.strftime('%Y-%m-%d') if existing_date else None

                        if (owner != existing_owner or
                            president != existing_president or
                            head_coach != existing_head_coach or
                            general_manager != existing_general_manager):
                            should_insert = True
                    else:
                        should_insert = True # No existing record, so insert

                    if should_insert:
                        insert_statement = text("""
                            INSERT INTO personnel (team_id, owner, president, head_coach, general_manager, extraction_date)
                            VALUES (:team_id, :owner, :president, :head_coach, :general_manager, :extraction_date)
                        """)
                        connection.execute(insert_statement, {
                            'team_id': team_id,
                            'owner': owner,
                            'president': president,
                            'head_coach': head_coach,
                            'general_manager': general_manager,
                            'extraction_date': extraction_date
                        })
                connection.commit()
                print(f"Successfully processed personnel data for {len(personnel_df_with_ids)} teams.")
        else:
            print("No personnel data to push (or no matching teams found).")
    except Exception as e:
        print(f"Error pushing personnel to SQL: {e}")


### Task 2.8: Implement `push_championships_to_sql`

Create a Python function named `push_championships_to_sql` that takes a Pandas DataFrame (`championships_df`) and a SQLAlchemy engine (`sql_engine`) as input. This function should:
1.  Retrieve `team_id` and `team_name` from the `teams` table in the database.
2.  Merge the `championships_df` with the retrieved team information to add the `team_id`.
3.  Push the DataFrame to the `championships` table in the MySQL database.
4.  Handle potential duplicates by updating existing records for a given `team_id` if the championship counts have changed, or insert a new one if no record exists.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `push_championships_to_sql`:

**Step 1: Retrieve `team_id`s from the Database**
* Use `pd.read_sql('SELECT team_id, team_name FROM teams', sql_engine)` to get the necessary team IDs.
* Merge this DataFrame with your input `championships_df` on `team_name` to associate each championship record with its corresponding `team_id`.

**Step 2: Implement Upsert Logic (Update or Insert)**
* Iterate through each row of the merged `championships_df`.
* For each row, construct an `INSERT` statement with `ON DUPLICATE KEY UPDATE` clauses for `league_championships`, `conference_championships`, and `division_championships`.
* This will insert a new record if `team_id` doesn't exist, or update the championship counts if `team_id` already exists.
* Execute this SQL statement using `connection.execute(text(...))` within a database connection.

**Step 3: Commit Changes**
* After iterating through all rows, commit the changes to the database using `connection.commit()`.
</details>

##### Solution

In [ ]:
def push_championships_to_sql(championships_df, sql_engine):
    """
    Pushes the championship DataFrame to the 'championships' table.
    Retrieves team_ids from the 'teams' table to establish foreign key links.
    Updates existing records for a given team_id or inserts new ones.
    """
    if championships_df.empty:
        print("No championship data to push to 'championships' table.")
        return

    try:
        teams_from_db = pd.read_sql('SELECT team_id, team_name FROM teams', sql_engine)
        championships_df_with_ids = pd.merge(championships_df,
                                             teams_from_db,
                                             left_on='team_name',
                                             right_on='team_name',
                                             how='inner')

        championships_df_with_ids.dropna(subset=['team_id'], inplace=True)
        championships_df_with_ids['team_id'] = championships_df_with_ids['team_id'].astype(int)

        if not championships_df_with_ids.empty:
            with sql_engine.connect() as connection:
                for index, row in championships_df_with_ids.iterrows():
                    team_id = row['team_id']
                    league_championships = row.get('league_championships')
                    conference_championships = row.get('conference_championships')
                    division_championships = row.get('division_championships')

                    sql_statement = text("""
                        INSERT INTO championships (team_id, league_championships, conference_championships, division_championships)
                        VALUES (:team_id, :league_championships, :conference_championships, :division_championships)
                        ON DUPLICATE KEY UPDATE
                        league_championships = VALUES(league_championships),
                        conference_championships = VALUES(conference_championships),
                        division_championships = VALUES(division_championships)
                    """)
                    connection.execute(sql_statement, {
                        'team_id': team_id,
                        'league_championships': league_championships,
                        'conference_championships': conference_championships,
                        'division_championships': division_championships
                    })
                connection.commit()
                print(f"Successfully pushed/updated {len(championships_df_with_ids)} championship records to 'championships' table.")
        else:
            print("No championship data to push (or no matching teams found).")
    except Exception as e:
        print(f"Error pushing championships to SQL: {e}")


## Part 3: Implementing Main Workflow Functions

Now, you will implement the main functions that orchestrate the entire data pipeline: scraping data for teams, personnel, and championships, and then storing them in the database. These functions will leverage the helper functions you just created.

### Task 3.1: Implement `scrap_and_store_teams_and_conferences`

Create a Python function named `scrap_and_store_teams_and_conferences` that takes `sql_engine` as input. This function should:
1.  Call `get_nfl_teams_from_wiki` to scrape the initial team, conference, and division data.
2.  Extract the unique team names from the scraped data to prepare for the `teams` table.
3.  Push these unique team names to the `teams` table.
4.  After the `teams` table is populated (and `team_id`s are generated), merge the original scraped data with the `team_id`s from the database.
5.  Prepare a DataFrame for the `conferences` table with `conference`, `division`, and `team_id`.
6.  Push this `conferences` DataFrame to the `conferences` table using `push_conferences_to_sql`.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `scrap_and_store_teams_and_conferences`:

**Step 1: Scrape Initial NFL Teams Data**
* Call `get_nfl_teams_from_wiki()` to get the DataFrame containing 'conference', 'division', and 'team_name'. Let's call this `initial_nfl_data_df`.

**Step 2: Prepare and Push to `teams` table**
* Create a new DataFrame for the `teams` table, containing only the unique `team_name` values from `initial_nfl_data_df`.
* Call `push_teams_to_sql(teams_to_push_df, sql_engine)`.

**Step 3: Retrieve `team_id`s and Prepare `conferences` table data**
* After pushing to `teams`, retrieve the `team_id`s and `team_name`s from the `teams` table in the database using `pd.read_sql`.
* Merge `initial_nfl_data_df` with this `teams_from_db_df` on the team name to associate `team_id`s with conference and division data.
* Create a `conferences_df` with 'conference', 'division', and 'team_id' columns. Ensure the column names match the SQL schema.

**Step 4: Push to `conferences` table**
* Call `push_conferences_to_sql(conferences_df, sql_engine)`.
</details>

##### Solution

In [ ]:
def scrap_and_store_teams_and_conferences(sql_engine):
    """
    Scrapes initial NFL team data (including conference and division),
    stores unique teams in the 'teams' table, and then populates the 'conferences' table.
    """
    print("Scraping initial NFL teams and conference data from Wikipedia...")
    initial_nfl_data_df = get_nfl_teams_from_wiki()

    if initial_nfl_data_df.empty:
        print("No initial NFL team data scraped. Skipping database operations for teams and conferences.")
        return

    all_teams_static_data = []
    # Filter out any None or empty team_name values before iterating
    valid_team_names = [name for name in initial_nfl_data_df['team_name'].unique() if name and str(name).strip() != '']

    for team_name in valid_team_names:
        static_info = get_static_team_data(team_name)
        all_teams_static_data.append(static_info)
    static_teams_df = pd.DataFrame(all_teams_static_data)

    # Ensure 'team_name' column in static_teams_df is also clean and not empty
    static_teams_df = static_teams_df.dropna(subset=['team_name'])
    static_teams_df = static_teams_df[static_teams_df['team_name'].astype(str).str.strip() != ''].copy()

    push_teams_to_sql(teams_to_push_df, sql_engine)

    teams_from_db_df = pd.read_sql('SELECT team_id, team_name FROM teams', sql_engine)

    # Merge initial_nfl_data_df with teams_from_db_df to get team_id before passing to push_conferences_to_sql
    conferences_df = pd.merge(initial_nfl_data_df,
                              teams_from_db_df,
                              on='team_name',
                              how='inner')
    # Select only the columns relevant for the conferences table AFTER the merge
    conferences_df = conferences_df[['conference', 'division', 'team_id']].copy()

    push_conferences_to_sql(conferences_df, sql_engine)


### Task 3.2: Implement `scrap_and_store_current_personnel`

Create a Python function named `scrap_and_store_current_personnel` that takes `sql_engine` as input. This function should:
1.  Use the provided SQLAlchemy engine.
2.  Read the list of `team_name`s directly from the `teams` table in your database.
3.  For each team, call `get_personnel_data` to scrape its current personnel information.
4.  Collect all scraped data into a Pandas DataFrame.
5.  Call `push_personnel_to_sql` to store the DataFrame in the `personnel` table.

**Note on Data Handling:** This function is designed to intelligently update or append new personnel records, preserving historical data only when actual personnel changes occur.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `scrap_and_store_current_personnel`:

**Step 1: Get Team Names from Database**
* Use `pd.read_sql('SELECT team_name FROM teams', sql_engine)` to get the list of team names.

**Step 2: Initialize Data Collection List**
* Create an empty list (e.g., `personnel_data_list = []`) to store the dictionaries returned by `get_personnel_data`.

**Step 3: Loop and Scrape Personnel Data**
* Iterate through each `team_name` in the retrieved list.
* Inside the loop, call `get_personnel_data(team_name)` to fetch the current personnel information for that team.
* Append the returned dictionary to your `personnel_data_list`.

**Step 4: Create DataFrame**
* After the loop, convert the `personnel_data_list` into a Pandas DataFrame (e.g., `nfl_personnel_df = pd.DataFrame(personnel_data_list)`).

**Step 5: Push to SQL**
* Call your `push_personnel_to_sql` helper function, passing the newly created `nfl_personnel_df` and the `sql_engine`.
</details>

##### Solution

In [ ]:
def scrap_and_store_current_personnel(sql_engine):
    """
    Scrapes current personnel data for all teams from Wikipedia and stores it in the 'personnel' table.
    Teams are retrieved from the database to ensure correct team_ids.
    """
    try:
        # Read team names directly from the 'teams' table
        teams_from_db_df = pd.read_sql('SELECT team_name FROM teams', sql_engine)
        list_of_teams = teams_from_db_df['team_name'].tolist()

        if not list_of_teams:
            print("No teams found in the database. Skipping personnel data scraping.")
            return

        personnel_data_list = []
        for team_name in list_of_teams:
            personnel_data = get_personnel_data(team_name)
            personnel_data_list.append(personnel_data)

        nfl_personnel_df = pd.DataFrame(personnel_data_list)
        push_personnel_to_sql(nfl_personnel_df, sql_engine)
        print("Current NFL Personnel data scraped and stored successfully!")
    except Exception as e:
        print(f"Error in scraping and storing personnel data: {e}")


### Task 3.3: Implement `scrap_and_store_current_championships`

Create a Python function named `scrap_and_store_current_championships` that takes `sql_engine` as input. This function should:
1.  Use the provided SQLAlchemy engine.
2.  Read the list of `team_name`s directly from the `teams` table in your database.
3.  For each team, call `get_championship_data` to scrape its current championship information.
4.  Collect all scraped data into a Pandas DataFrame.
5.  Call `push_championships_to_sql` to store the DataFrame in the `championships` table.

**Note on Data Handling:** This function is designed to **update** existing records for a given `team_id` if the championship counts have changed, or insert a new one if no record exists. This ensures that championship totals are always current, reflecting the latest information from Wikipedia, and historical `extraction_year` data is NOT maintained in this table.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Here's a step-by-step guide to help you implement `scrap_and_store_current_championships`:

**Step 1: Get Team Names from Database**
* Use `pd.read_sql('SELECT team_name FROM teams', sql_engine)` to get the list of team names.

**Step 2: Initialize Data Collection List**
* Create an empty list (e.g., `championship_data_list = []`) to store the dictionaries returned by `get_championship_data`.

**Step 3: Loop and Scrape Championship Data**
* Iterate through each `team_name` in the retrieved list.
* Inside the loop, call `get_championship_data(team_name)` to fetch the current championship information for that team.
* Append the returned dictionary to your `championship_data_list`.

**Step 4: Create DataFrame**
* After the loop, convert the `championship_data_list` into a Pandas DataFrame (e.g., `nfl_championships_df = pd.DataFrame(championship_data_list)`).

**Step 5: Push to SQL**
* Call your `push_championships_to_sql` helper function, passing the newly created `nfl_championships_df` and the `sql_engine`.
</details>

##### Solution

In [ ]:
def scrap_and_store_current_championships(sql_engine):
    """
    Scrapes current championship data for all teams from Wikipedia and stores it in the 'championships' table.
    Teams are retrieved from the database to ensure correct team_ids.
    """
    try:
        # Read team names directly from the 'teams' table
        teams_from_db_df = pd.read_sql('SELECT team_name FROM teams', sql_engine)
        list_of_teams = teams_from_db_df['team_name'].tolist()

        if not list_of_teams:
            print("No teams found in the database. Skipping championship data scraping.")
            return

        championship_data_list = []
        for team_name in list_of_teams:
            championship_data = get_championship_data(team_name)
            championship_data_list.append(championship_data)

        nfl_championships_df = pd.DataFrame(championship_data_list)

        push_championships_to_sql(nfl_championships_df, sql_engine)
        print("Current NFL Championships data scraped and stored successfully!")
    except Exception as e:
        print(f"Error in scraping and storing championship data: {e}")


## Part 4: Executing the Data Pipeline

Now that all helper and main workflow functions are defined, it's time to execute the data pipeline. You will set up the database connection, and then call the main scraping and storage functions in sequence.

### Task 4.1: Establish MySQL Connection and Create Tables

**Your Task:** Establish a connection to your MySQL database using `sqlalchemy`. Define the connection parameters (schema, host, user, password, port) and create the database engine. Then, ensure all necessary tables (`teams`, `conferences`, `personnel`, `championships`) exist in your database. You can choose one of the two methods below to create your tables.

#### Option A: Manual MySQL Client Setup

If you prefer to manage your database schema directly, you can use a MySQL client (like MySQL Workbench or the command line) to create the `sql_workshop` database and the necessary tables. After running the SQL commands in your client, you will then define the SQLAlchemy engine in Python to connect to this pre-existing database.

##### Your Task (Part 1): Manually create the database and all necessary tables in your MySQL client.


In [ ]:
# No Python code needed here for manual setup. Execute SQL in your MySQL client.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Connect to your MySQL server using your preferred client. Then, copy and paste the SQL commands from the solution section below and execute them. Ensure there are no errors during execution.

After successful execution, you can verify the tables exist by running `SHOW TABLES;` within the `sql_workshop` database in your MySQL client.
</details>

##### Solution (for Option A - Part 1: SQL Commands)
<details>
<summary>Click to reveal solution</summary>

Run the following SQL commands in your MySQL client:
```sql
CREATE DATABASE IF NOT EXISTS sql_workshop;
USE sql_workshop;

CREATE TABLE IF NOT EXISTS teams (
    team_id INT PRIMARY KEY AUTO_INCREMENT,
    team_name VARCHAR(255) NOT NULL UNIQUE,
    city VARCHAR(255),
    state VARCHAR(255),
    stadium VARCHAR(255),
    established_year INT,
    colors TEXT,
    mascot VARCHAR(255),
    website VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS conferences (
    team_id INT PRIMARY KEY,
    conference VARCHAR(255),
    division VARCHAR(255),
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);

CREATE TABLE IF NOT EXISTS personnel (
    personnel_id INT PRIMARY KEY AUTO_INCREMENT,
    team_id INT,
    owner TEXT,
    president TEXT,
    head_coach TEXT,
    general_manager TEXT,
    extraction_date DATE,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);

CREATE TABLE IF NOT EXISTS championships (
    team_id INT PRIMARY KEY,
    league_championships INT,
    conference_championships INT,
    division_championships INT,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);
```
After executing the above SQL, your database schema will be ready. Proceed to the next part to define the Python engine.
</details>

##### Your Task (Part 2): Now that the database and tables are manually created, define the connection parameters and establish the SQLAlchemy engine in Python to connect to the `sql_workshop` database.


##### Hint
<details>
<summary>Click to reveal hint</summary>

Define the `schema`, `host`, `user`, `password` (from `keys.MySQL_pass`), and `port` variables.
Construct the database connection string using an f-string.
Use `create_engine()` from `sqlalchemy`.
Include a `try-except` block to handle `OperationalError` for connection issues.
You can test the connection with a simple query like `connection.execute(text("SELECT 1"))`.
</details>

##### Solution

In [ ]:
schema = "sql_workshop"
host = "127.0.0.1"
user = "root"
password = keys.MySQL_pass
port = 3306

db_connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

engine = None
try:
    engine = create_engine(db_connection_string)
    with engine.connect() as connection:
        connection.execute(text("SELECT 1"))
    print("SQLAlchemy engine for 'sql_workshop' created and connected successfully.")

except OperationalError as e:
    print(f"Error connecting to MySQL database: {e}")
    print("Please ensure your MySQL server is running, 'sql_workshop' database exists, and credentials are correct.")
except Exception as e:
    print(f"An unexpected error occurred during engine creation or connection test: {e}")

#### Option B: Python `connection.execute` Setup

If you prefer to automate the database and table creation process using Python, you can define and call a function that uses SQLAlchemy's `connection.execute` to run the `CREATE DATABASE IF NOT EXISTS` and `CREATE TABLE IF NOT EXISTS` commands. This approach ensures your schema is set up programmatically.

**Your Task:** Define the connection parameters, establish the SQLAlchemy engine (including creating the database if it doesn't exist), and then define and call the `create_nfl_tables_mysql` function to create the necessary tables in your database using Python.


##### Hint
<details>
<summary>Click to reveal hint</summary>

Define the `schema`, `host`, `user`, `password`, and `port` variables. Then, create two connection strings: one for the server (`server_connection_string`) and one for the specific database (`db_connection_string`).

Use a `try-except` block to first connect to the server (using `server_connection_string`) and execute `CREATE DATABASE IF NOT EXISTS {schema};`. After that, create the main `engine` using `db_connection_string`.

Define a function, for example, `create_nfl_tables_mysql(engine)`, that encapsulates the SQL `CREATE TABLE IF NOT EXISTS` statements. Inside this function, use a `with engine.connect() as connection:` block to execute each SQL command using `connection.execute(text(sql_command))` and `connection.commit()`.

Finally, call the `create_nfl_tables_mysql(engine)` function.

Ensure `team_id` is set as the `PRIMARY KEY` for the `championships` and `conferences` tables.
</details>

##### Solution (for Option B)


In [ ]:
schema = "sql_workshop"
host = "127.0.0.1"
user = "root"
password = keys.MySQL_pass
port = 3306

server_connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/'
db_connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

engine = None

try:
    server_engine = create_engine(server_connection_string)
    with server_engine.connect() as connection:
        connection.execute(text(f"CREATE DATABASE IF NOT EXISTS {schema};"))
        connection.commit()
    print(f"Database '{schema}' created or already exists.")

    engine = create_engine(db_connection_string)

except OperationalError as e:
    print(f"Error connecting to MySQL server or database: {e}")
    print("Please ensure your MySQL server is running and connection details (host, port, user, password) are correct.")
except Exception as e:
    print(f"An unexpected error occurred during database creation or engine creation: {e}")

def create_nfl_tables_mysql(engine):
    """
    Ensures the 'teams', 'conferences', and 'championships' tables exist in the MySQL database.
    """
    if engine is None:
        print("Database engine not initialized. Cannot create tables.")
        return

    try:
        with engine.connect() as connection:
            sql_commands = [
                """
                CREATE TABLE IF NOT EXISTS teams (
                    team_id INT PRIMARY KEY AUTO_INCREMENT,
                    team_name VARCHAR(255) NOT NULL UNIQUE,
                    city VARCHAR(255),
                    state VARCHAR(255),
                    stadium VARCHAR(255),
                    established_year INT,
                    colors TEXT,
                    mascot VARCHAR(255),
                    website VARCHAR(255)
                );
                """,
                """
                CREATE TABLE IF NOT EXISTS conferences (
                    team_id INT PRIMARY KEY,
                    conference VARCHAR(255),
                    division VARCHAR(255),
                    FOREIGN KEY (team_id) REFERENCES teams(team_id)
                );
                """,
                """
                CREATE TABLE IF NOT EXISTS personnel (
                    personnel_id INT PRIMARY KEY AUTO_INCREMENT,
                    team_id INT,
                    owner TEXT,
                    president TEXT,
                    head_coach TEXT,
                    general_manager TEXT,
                    extraction_date DATE,
                    FOREIGN KEY (team_id) REFERENCES teams(team_id)
                );
                """,
                """
                CREATE TABLE IF NOT EXISTS championships (
                    team_id INT PRIMARY KEY,
                    league_championships INT,
                    conference_championships INT,
                    division_championships INT,
                    FOREIGN KEY (team_id) REFERENCES teams(team_id)
                );
                """
            ]

            for cmd in sql_commands:
                connection.execute(text(cmd))
                connection.commit()
            print("All NFL tables prepared (or confirmed) using Python.")
    except OperationalError as e:
        print(f"Error during table creation: {e}")
        print("Please ensure you have the necessary permissions for the database and tables.")
    except ProgrammingError as e:
        print(f"SQL syntax error during table creation: {e}")
        print("Please review the SQL schema for any syntax issues.")
    except Exception as e:
        print(f"An unexpected error occurred during table creation: {e}")

if engine is not None:
    create_nfl_tables_mysql(engine)
else:
    print("Skipping table creation due to prior database connection error.")

### Task 4.2: Execute `scrap_and_store_teams_and_conferences`

Call the `scrap_and_store_teams_and_conferences` function to begin scraping initial team, conference, and division data and storing it in your database.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Call the function you defined in Task 3.1, passing the `sql_engine` (which is the `engine` variable from Task 4.1).

```python
scrap_and_store_teams_and_conferences(engine)
```
</details>

##### Solution

In [ ]:
print("\n--- Starting Initial Team & Conference Data Scraping and Storage ---")
if 'engine' in locals() and engine is not None:
    scrap_and_store_teams_and_conferences(engine)
else:
    print("Database engine not initialized. Skipping initial team and conference data scraping.")
print("--- Initial Team & Conference Data Scraping and Storage Complete! ---")

### Task 4.3: Execute `scrap_and_store_current_personnel`

Now, call the `scrap_and_store_current_personnel` function to scrape and store the latest personnel data for all teams. This function will retrieve the list of team names directly from the database to ensure data consistency.

**Note on Data Handling:** The `push_personnel_to_sql` function is designed to intelligently update or append new personnel records, preserving historical data only when actual personnel changes occur.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Call the function you defined in Task 3.2, passing the `sql_engine`.
The function will internally retrieve the list of teams from the database.

```python
scrap_and_store_current_personnel(engine)
```
</details>

##### Solution

In [ ]:
print("\n--- Starting Personnel Data Scraping and Storage ---")
if 'engine' in locals() and engine is not None:
    scrap_and_store_current_personnel(engine)
else:
    print("Database engine not initialized. Skipping personnel data scraping.")
print("--- Personnel Data Scraping and Storage Complete! ---")

### Task 4.4: Execute `scrap_and_store_current_championships`

Finally, call the `scrap_and_store_current_championships` function to scrape and store the latest championship data. This will complete the data acquisition and storage pipeline.

**Note on Data Handling:** This function is designed to **update** existing records for a given `team_id` if the championship counts have changed, or insert a new one if no record exists. This ensures that championship totals are always current, reflecting the latest information from Wikipedia, and historical `extraction_year` data is NOT maintained in this table.

##### Hint
<details>
<summary>Click to reveal hint</summary>

Call the function you defined in Task 3.3, passing the `sql_engine`.
The function will internally retrieve the list of teams from the database.

```python
scrap_and_store_current_championships(engine)
```
</details>

##### Solution

In [ ]:
print("\n--- Starting Championships Data Scraping and Storage ---")
if 'engine' in locals() and engine is not None:
    scrap_and_store_current_championships(engine)
else:
    print("Database engine not initialized. Skipping championships data scraping.")
print("--- Championships Data Scraping and Storage Complete! ---")

## Congratulations!
You've successfully completed the NFL Data Science Workshop. You've learned to scrape data from the web, organize it with Pandas, and integrate it into a MySQL database, setting the foundation for further data analysis and insights!